<a href="https://colab.research.google.com/github/athulskrish/pytorch_nn/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use this example - https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/ to understand pytorch NNs. Augment the dataset by perturbing the data fields with gaussian noise. Train/test/validate. Take a screencast walkthrough of the full code, including your understanding of how the NN is implemented.

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('/content/drive/MyDrive/datasets/pima-indians-diabetes.csv', delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)


print(dataset)

[[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  1.     85.     66.    ...   0.351  31.      0.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 ...
 [  5.    121.     72.    ...   0.245  30.      0.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]
 [  1.     93.     70.    ...   0.315  23.      0.   ]]


In [4]:

# define the model
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

model = PimaClassifier()
print(model)


PimaClassifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [5]:

# train the model
loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [6]:

# compute accuracy
y_pred = model(X)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy*100}")

# make class predictions with the model
predictions = (model(X) > 0.5).int()
for i in range(5):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

Accuracy 71.09375
[6.0, 148.0, 72.0, 35.0, 0.0, 33.599998474121094, 0.6269999742507935, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.600000381469727, 0.35100001096725464, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.299999237060547, 0.671999990940094, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.100000381469727, 0.16699999570846558, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.099998474121094, 2.2880001068115234, 33.0] => 1 (expected 1)


Including train,test, validate

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y, stratify=y, test_size=0.2,random_state=66)


Including validation as 25% of the training data set

In [9]:
X_train,X_val, y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25, random_state=66)

INcluding noise

In [14]:
import random
X_deviation=0.01
noise= torch.randn(X.size())*X_deviation

X=X+noise

print(X[:5])


tensor([[  5.9917, 147.9850,  71.9999,  ...,  33.5950,   0.6298,  49.9936],
        [  1.0253,  84.9890,  66.0227,  ...,  26.6114,   0.3494,  31.0042],
        [  8.0011, 183.0208,  64.0180,  ...,  23.3040,   0.6643,  32.0180],
        ...,
        [  4.9922, 120.9970,  72.0067,  ...,  26.2028,   0.2244,  29.9997],
        [  1.0022, 125.9904,  60.0048,  ...,  30.0945,   0.3484,  47.0025],
        [  0.9976,  92.9765,  69.9959,  ...,  30.4076,   0.3105,  22.9779]])
tensor([[ 5.9910e+00,  1.4798e+02,  7.2004e+01,  3.5002e+01, -2.8754e-02,
          3.3599e+01,  6.4469e-01,  4.9997e+01],
        [ 1.0141e+00,  8.5001e+01,  6.6030e+01,  2.8991e+01,  1.7581e-03,
          2.6619e+01,  3.5018e-01,  3.1011e+01],
        [ 8.0091e+00,  1.8302e+02,  6.4021e+01, -1.9484e-02, -2.5089e-02,
          2.3290e+01,  6.8077e-01,  3.2019e+01],
        [ 1.0058e+00,  8.9000e+01,  6.5997e+01,  2.2990e+01,  9.4009e+01,
          2.8102e+01,  1.6949e-01,  2.1010e+01],
        [ 9.5807e-03,  1.3698e+02,  3.